<a href="https://colab.research.google.com/github/Dewanik/RNN_from_scratch/blob/master/Recurrent_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Reccurent Neural Network from scratch**
**Understanding the mathematics behind RNNs**

Importing Numpy for matrix math

In [0]:
import copy, numpy as np

**Reading Data From Kafka.txt creating a dictinoary map of characters to integer(or their position) in total number of unique characters in data and
and integer(or their position) in total number of data to its accosiated characters and one_hot_encoding of the character**

In [0]:
data = open('kafka.txt','r').read()
chars = list(set(data))
n = 2
data_size , unique_size = len(data),len(chars)
char_ix = {ch:i for i,ch in enumerate(chars)}
ix_char = {i:ch for i,ch in enumerate(chars)}



Weight matrix Initialization for : 


1.   Input to Current Hidden Layer 
2.   Current Hidden to Previous Hidden Layer
3.   Merged Current Hidden to output

and biases 




In [0]:
hidden_size = 100
seq_length = 500

wxh = np.random.randn(hidden_size, len(chars)) * 0.01
whh = np.random.randn(hidden_size, hidden_size) * 0.01
why = np.random.randn(len(chars), hidden_size) * 0.01 #input to hidden
bh = np.zeros((hidden_size,1))
by = np.zeros((len(chars),1))




In [0]:
print(wxh.shape)

(100, 80)


Example of One_Hot_Encoding Of Character to Vectors

In [0]:
vector_for_a = np.zeros((len(chars),1))
vector_for_a[char_ix['a']] = 1


Feed Forward of RNN 

Creating a dataset of the position of characters mapping it to next upcoming character in kafka.txt dataset

In [5]:

    

def train(x,y,hprev):
    xs , hs , ys , ps = {}, {} , {} , {}
    hs[-1] = np.copy(hprev)
    loss = 0
    
    #Forward Propagation
    for i in range(seq_length):
        
        xs[i] = np.zeros((len(chars),1))
        xs[i][x[i]] = 1
        hs[i] = np.tanh(np.dot(wxh,xs[i]) + np.dot(whh,hs[i-1]) + bh)
        
        ys[i] = np.dot(why,hs[i])+ by
        ps[i] = np.exp(ys[i])/np.sum(np.exp(ys[i]))
       
        loss += -np.log(ps[i][y[i]])
        
    
    #Back Propagation Through Time (BPTT)
    dwxh , dwhh , dwhy , dbh , dby = np.zeros_like(wxh) , np.zeros_like(whh) , np.zeros_like(why) , np.zeros_like(bh) ,np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for i in reversed(range(len(x))):
        #Updating Gradients calculating small gradients at every time step
        dy = np.copy(ps[i])
        dy[y[i]] -= 1
        
        dwhy += np.dot(dy,hs[i].T)
        
        dby += dy
        
        
        dh = np.dot(why.T,dy) + dhnext
        
        dhraw = (1 - hs[i] * hs[i]) * dh
        
        dbh +=  dhraw
        dwxh += np.dot(dhraw,xs[i].T)
        dwhh += np.dot(dhraw,hs[i-1].T)
       
        dhnext = np.dot(whh.T,dhraw)
        
    for dparam in [dwxh, dwhh, dwhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients                                                                                                                 
    return loss, dwxh, dwhh, dwhy, dbh, dby, hs[seq_length-1]
        
def predict(seed_x,h):
    
    x = np.zeros((len(chars),1))
    x[seed_x] = 1
    vec_outputs = list()
    yguess = list()
    
    for i in range(seq_length):
        #Input * whx +  prev_hidden * whh + bh ==> new_merged_hidden
        h = np.tanh(np.dot(wxh,x) + np.dot(whh,h) + bh)
        
        #new_merged_hidden * why + by ==> yhat
        y = np.dot(why,h) + by
        
        #normalized(yhat) ==> normalized_output
        p = np.exp(y)/np.sum(np.exp(y))
       
        #new_vecx == normalized_output == new_x
        #Choose with highest probability
        ix = np.argmax(p)
        x = np.zeros((len(chars),1))
        x[ix] = 1
        yguess.append(ix)
        
    
    
    print(''.join([ix_char[c] for c in yguess]))

x = [char_ix[c] for c in data[0:seq_length]]
hprev = np.zeros((hidden_size,1))
predict(x[0],hprev)

J!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m'cGNJ!m


In [0]:
   
x = [char_ix[c] for c in data[0:seq_length]]
y = [char_ix[c] for c in data[1:seq_length+1]]
print('Given X:')
print(x[0])
print(ix_char[x[0]])
print('Known Y:')
print(y)
print(len(y))
print(''.join([ix_char[c] for c in y]))

print('\n')



nses , pses = 0,0
lr = 1e-1
mWxh, mWhh, mWhy = np.zeros_like(wxh), np.zeros_like(whh), np.zeros_like(why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad                                                                                                                
smooth_loss = -np.log(1.0/len(chars))*seq_length # loss at iteration 0 

for i in range(1000*100):
    if pses+seq_length+1 >= len(data) or nses == 0:
        hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
        pses = 0 # go from start of data
    x = [char_ix[c] for c in data[0:seq_length]]
    y = [char_ix[c] for c in data[1:seq_length+1]]
    loss, dwxh, dwhh, dwhy, dbh, dby, hprev  = train(x,y,hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if i % 1000 == 0:
        predict(x[0],hprev)
        print('Iter:{1}   Loss:{0}'.format(smooth_loss,i))
    for param, dparam, mem in zip([wxh, whh, why, bh, by],
                                [dwxh, dwhh, dwhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -lr * dparam / np.sqrt(mem + 1e-8) # adagrad update
    

    
    



Given X:
45
O
Known Y:
[34, 22, 48, 4, 51, 31, 34, 47, 34, 37, 74, 48, 66, 7, 22, 34, 48, 57, 31, 22, 37, 51, 31, 48, 72, 46, 4, 15, 46, 48, 66, 51, 14, 22, 48, 20, 31, 51, 4, 48, 32, 31, 51, 11, 64, 2, 22, 78, 48, 78, 31, 22, 46, 4, 15, 74, 48, 7, 22, 48, 20, 51, 11, 34, 78, 48, 7, 47, 4, 15, 22, 2, 20, 48, 32, 31, 46, 34, 15, 20, 51, 31, 4, 22, 78, 48, 47, 34, 48, 7, 47, 15, 48, 64, 22, 78, 48, 47, 34, 32, 51, 48, 46, 48, 7, 51, 31, 31, 47, 64, 2, 22, 48, 52, 22, 31, 4, 47, 34, 63, 48, 43, 22, 48, 2, 46, 30, 48, 51, 34, 48, 7, 47, 15, 48, 46, 31, 4, 51, 11, 31, 79, 2, 47, 14, 22, 48, 64, 46, 71, 14, 74, 48, 46, 34, 78, 48, 47, 20, 48, 7, 22, 48, 2, 47, 20, 32, 22, 78, 48, 7, 47, 15, 48, 7, 22, 46, 78, 48, 46, 48, 2, 47, 32, 32, 2, 22, 48, 7, 22, 48, 71, 51, 11, 2, 78, 48, 15, 22, 22, 48, 7, 47, 15, 48, 64, 31, 51, 66, 34, 48, 64, 22, 2, 2, 30, 74, 48, 15, 2, 47, 37, 7, 32, 2, 30, 48, 78, 51, 4, 22, 78, 48, 46, 34, 78, 48, 78, 47, 52, 47, 78, 22, 78, 48, 64, 30, 48, 46, 31, 71, 7, 22,

KeyboardInterrupt: ignored

In [0]:
length = 600

x = [char_ix[c] for c in data[506:seq_length]]
y = [char_ix[c] for c in data[507:seq_length+1]]
print(ix_char[x[1]])
print(y)
hprev = np.zeros((hidden_size,1))
predict(x[1],hprev)